## Knowledge Graphs

Knowledge Graphs, a form of graph-based knowledge representation, provide a method for modeling and storing interlinked information in a format that is both human- and machine-understandable. These graphs consist of *nodes* and *edges*, representing entities and their relationships. Unlike traditional databases, the inherent expressiveness of graphs allows for richer semantic understanding, while providing the flexibility to accommodate new entity types and relationships without being constrained by a fixed schema.

By combining knowledge graphs with embeddings (vector search), we can leverage *multi-hop connectivity* and *contextual understanding of information* to enhance querying, reasoning, and explainability in LLMs. This notebook explores the practical implementation of this approach, demonstrating how to (i) build a knowledge graph of academic publications, and (ii) extract actionable insights from it.

<p align="center">
  <img src="./static/knowledge-graphs.png">
</p>

### 1. Knowledge Graph Initialization

We will create our Knowledge Graph using [Neo4j](https://neo4j.com/), an open-source database management system that specializes in graph database technology.

In [1]:
%pip install neo4j langchain langchain_openai langchain-community python-dotenv --quiet | tail -n 1

Note: you may need to restart the kernel to use updated packages.


#### 1.1 Setting Up a Neo4j Instance

For a quick and easy setup, you can start a free instance on [Neo4j Aura](https://neo4j.com/product/auradb/). 

In [2]:
import dotenv
dotenv.load_dotenv('.env', override=True)

True

In [4]:
import os
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=os.environ['NEO4J_URI'], 
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD'],
)

#### 1.2 Loading Dataset into a Graph

The below example creates a connection with our Neo4j database and populates it with synthetic data about research articles and their authors. 

The entities are: 
- *Researcher*
- *Article*
- *Topic*

Whereas the relationships are:
- *Researcher* --[PUBLISHED]--> *Article*
- *Article* --[IN_TOPIC]--> *Topic*



In [5]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

q_load_articles = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/dcarpintero/generative-ai-101/main/dataset/synthetic_articles.csv' 
AS row 
FIELDTERMINATOR ';'
MERGE (a:Article {title:row.Title})
SET a.abstract = row.Abstract,
    a.publication_date = date(row.Publication_Date)
FOREACH (researcher in split(row.Authors, ',') | 
    MERGE (p:Researcher {name:trim(researcher)})
    MERGE (p)-[:PUBLISHED]->(a))
FOREACH (topic in [row.Topic] | 
    MERGE (t:Topic {name:trim(topic)})
    MERGE (a)-[:IN_TOPIC]->(t))
"""

graph.query(q_load_articles)

[]

In [10]:
graph.refresh_schema()
print(graph.get_schema)

Node properties:
Article {title: STRING, abstract: STRING, publication_date: DATE}
Researcher {name: STRING}
Topic {name: STRING}
Relationship properties:

The relationships:
(:Article)-[:IN_TOPIC]->(:Topic)
(:Researcher)-[:PUBLISHED]->(:Article)


#### 1.3 Build Vector Index

We implement a vector index to efficiently search for relevant articles based on their *topic, title, and abstract*. This process involves calculating the embeddings for each article using these fields. At query time, the system finds the most similar articles to the user's input by employing a similarity metric, such as cosine distance.


In [11]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=os.environ['NEO4J_URI'],
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD'],
    index_name='articles',
    node_label="Article",
    text_node_properties=['topic', 'title', 'abstract'],
    embedding_node_property='embedding',
)

### 2. Q&A on Similarity

In [31]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)

In [32]:
r = vector_qa.invoke(
    {"query": "which articles are related to protecting users from misinformation? include the article titles and abstracts."}
)
print(r['result'])

The articles related to protecting users from misinformation are:

1. Title: Language Models for Automated News Article Generation
Abstract: We discuss the use of language models in generating news articles, exploring the benefits and ethical considerations related to media integrity.

2. Title: Language Models for Detecting Fake Reviews
Abstract: We propose language model-based approaches for detecting fake and misleading reviews online, helping users make informed decisions.


### 3. Graph-Cypher-Chain w/ LangChain

LangChain provides also wrapper around Neo4j graph database that allows to generate Cypher statements based on the user input and use them to retrieve relevant information from the database.

In [12]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4o'),
    qa_llm = ChatOpenAI(temperature=0, model_name='gpt-4o'), 
    graph=graph,
    verbose=True,
)

### 4. Inference traversing Knowledge Graphs

Knowledge graphs excel in their ability to query and navigate the connections between entities, allowing for the retrieval of pertinent information and the discovery of new insights.

#### 3.1 Sample 1

In this example, our question 'How many articles has published Emma Wilson' will be translated into the Cyper query:

```
MATCH (r:Researcher {name: "Emma Wilson"})-[:PUBLISHED]->(a:Article)
RETURN COUNT(a) AS numberOfArticles
```

which matches nodes labeled `Author` with the name 'Emma Wilson' and traverses the `PUBLISHED` relationships to `Article` nodes. 
It then counts the number of `Article` nodes connected to 'Emma Wilson':

In [13]:
# the answer should be '5'
cypher_chain.invoke(
    {"query": "How many articles has published Emma Wilson?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r:Researcher {name: "Emma Wilson"})-[:PUBLISHED]->(a:Article)
RETURN COUNT(a) AS numberOfArticles

Full Context:
[{'numberOfArticles': 5}]

> Finished chain.


{'query': 'How many articles has published Emma Wilson?',
 'result': 'Emma Wilson has published 5 articles.'}

#### 3.2 Sample 2

In this example the query 'are there any pair of researchers who have published more than one article together?' results in the Cypher query:

```
MATCH (r1:Researcher)-[:PUBLISHED]->(a:Article)<-[:PUBLISHED]-(r2:Researcher)
WHERE r1 <> r2
WITH r1, r2, COUNT(a) AS sharedArticles
WHERE sharedArticles > 1
RETURN r1.name, r2.name, sharedArticles
```

which results in traversing from the `Researcher` nodes to the `PUBLISHED` relationship to find connected `Article` nodes, and then traversing back to find `Researchers` pairs.

In [14]:
# the answer should be Alice Johnson and David Miller, Alexander Lee and David Miller, Olivia Taylor and Alexander Lee, and David Miller and Alice Johnson
cypher_chain.invoke(
    {"query": "are there any pair of researchers who have published more than one article together?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r1:Researcher)-[:PUBLISHED]->(a:Article)<-[:PUBLISHED]-(r2:Researcher)
WHERE r1 <> r2
WITH r1, r2, COUNT(a) AS sharedArticles
WHERE sharedArticles > 1
RETURN r1.name, r2.name, sharedArticles

Full Context:
[{'r1.name': 'Alice Johnson', 'r2.name': 'David Miller', 'sharedArticles': 2}, {'r1.name': 'Alexander Lee', 'r2.name': 'David Miller', 'sharedArticles': 2}, {'r1.name': 'Olivia Taylor', 'r2.name': 'Alexander Lee', 'sharedArticles': 2}, {'r1.name': 'David Miller', 'r2.name': 'Alexander Lee', 'sharedArticles': 2}, {'r1.name': 'Alexander Lee', 'r2.name': 'Olivia Taylor', 'sharedArticles': 2}, {'r1.name': 'David Miller', 'r2.name': 'Alice Johnson', 'sharedArticles': 2}]

> Finished chain.


{'query': 'are there any pair of researchers who have published more than one article together?',
 'result': 'Yes, there are pairs of researchers who have published more than one article together. These pairs are:\n\n- Alice Johnson and David Miller\n- Alexander Lee and David Miller\n- Olivia Taylor and Alexander Lee\n- David Miller and Alexander Lee\n- Alexander Lee and Olivia Taylor\n- David Miller and Alice Johnson'}

#### 3.3 Sample 3

It appears David Miller has collaborated with many peers. Lets find out is he is the researcher with most peers collaborations. 
Our query 'which researcher has collaborated with the most peers?' results now in the Cyper:

```
MATCH (r:Researcher)-[:PUBLISHED]->(:Article)<-[:PUBLISHED]-(peer:Researcher)
WITH r, COUNT(DISTINCT peer) AS peerCount
RETURN r.name AS researcher, peerCount
ORDER BY peerCount DESC
LIMIT 1
```

Here, we need to star from all `Researcher` nodes and traverse their `PUBLISHED` relationships to find connected `Article` nodes. For each `Article` node, Neo4j then traverses back to find other `Researcher` nodes (peer) who have also published the same article.

In [15]:
# the answer should be 'David Miller' with 5
cypher_chain.invoke(
    {"query": "Which researcher has collaborated with the most peers?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r1:Researcher)-[:PUBLISHED]->(:Article)<-[:PUBLISHED]-(r2:Researcher)
WHERE r1 <> r2
WITH r1, COUNT(DISTINCT r2) AS collaborators
RETURN r1.name AS researcher, collaborators
ORDER BY collaborators DESC
LIMIT 1

Full Context:
[{'researcher': 'David Miller', 'collaborators': 5}]

> Finished chain.


{'query': 'Which researcher has collaborated with the most peers?',
 'result': 'David Miller has collaborated with 5 peers.'}